In [1]:
!del /Q utils\__pycache__

In [2]:
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
from adtk.metrics import f1_score, precision, recall

In [4]:
from utils.datasets import GhlKasperskyDataset, TepHarvardDataset
from utils.watchmen import LimitWatchman, LimitPcaWatchman, SpePcaWatchman
from utils.custom_plots import plot_stacked

# Preparing datasets

## GHL

In [5]:
ghl = GhlKasperskyDataset()
ghl

GhlKasperskyDataset(E:\Datasets\GHL)

## TEP Harvard

In [6]:
tep1 = TepHarvardDataset()
tep1

TepHarvardDataset(E:\Datasets\TEP\dataverse)

## Bring them all

In [7]:
datasets = {
    0: ghl,
    1: tep1,
}

# Preparing watchmen

## LimitWatchman

In [8]:
limit_watchmen = {
    0: LimitWatchman(),
    1: LimitWatchman(ewma='3 min'),
}

## LimitPcaWatchman

In [9]:
limit_pca_watchmen = {
    0: LimitPcaWatchman(n_components=1),
    1: LimitPcaWatchman(n_components=12),
}

In [10]:
for k in datasets:
    datasets[k].shake_not_stir()
    for data, faults, info in tqdm(datasets[k].train_generator(), desc=f'fit scaler on dataset {k}'):
        limit_pca_watchmen[k].partial_fit_scaler(data)
    for data, faults, info in tqdm(datasets[k].train_generator(), desc=f'fit pca on dataset {k}'):
        limit_pca_watchmen[k].partial_fit_pca(data)        

fit scaler on dataset 0: 0it [00:00, ?it/s]

fit pca on dataset 0: 0it [00:00, ?it/s]

fit scaler on dataset 1: 0it [00:00, ?it/s]

fit pca on dataset 1: 0it [00:00, ?it/s]

## SpePcaWatchman

In [11]:
spe_pca_watchmen = {
    0: SpePcaWatchman(n_components=1),
    1: SpePcaWatchman(n_components=12),
}

In [12]:
for k in datasets:
    datasets[k].shake_not_stir()
    for data, _, _ in tqdm(datasets[k].train_generator(), desc=f'fit scaler on dataset {k}'):
        spe_pca_watchmen[k].partial_fit_scaler(data)
    for data, _, _ in tqdm(datasets[k].train_generator(), desc=f'fit pca on dataset {k}'):
        spe_pca_watchmen[k].partial_fit_pca(data)        

fit scaler on dataset 0: 0it [00:00, ?it/s]

fit pca on dataset 0: 0it [00:00, ?it/s]

fit scaler on dataset 1: 0it [00:00, ?it/s]

fit pca on dataset 1: 0it [00:00, ?it/s]

## Bring them all

In [13]:
watchmen = {
    0: limit_watchmen,
    1: limit_pca_watchmen,
    2: spe_pca_watchmen,
}

# Learning

In [14]:
for k in datasets:
    datasets[k].shake_not_stir()
    for w in watchmen:
        for data, _, _ in tqdm(datasets[k].train_generator(), desc=f'learn watchman {w} on dataset {k}'):
            watchmen[w][k].learn(data)

learn watchman 0 on dataset 0: 0it [00:00, ?it/s]

learn watchman 1 on dataset 0: 0it [00:00, ?it/s]

learn watchman 2 on dataset 0: 0it [00:00, ?it/s]

learn watchman 0 on dataset 1: 0it [00:00, ?it/s]

learn watchman 1 on dataset 1: 0it [00:00, ?it/s]

learn watchman 2 on dataset 1: 0it [00:00, ?it/s]

# Examine